# RuleBased Classification of QA dataset

In [ ]:
import polars as pl
import re
from typing import Any

In [ ]:
df_ar_mini = pl.read_parquet("df_ar_mini.parquet")

In [ ]:
print(f"Answerable proportion: {df_ar_mini['answerable'].sum() / df_ar_mini.height}")

In [ ]:
def when_digit_rule(question: str, context: str) -> bool:
    return ("when" in question.lower() and bool(re.search(r'\d', context)))

In [ ]:
# Should be true
print(when_digit_rule(df_ar_mini[0, "translation"], df_ar_mini[0, "context"]))
print(lambda x: x*2)

In [ ]:
def rule_based_classification(question: str, context: str, rules_list: list[Any]) -> bool:
    return bool(sum([rule(question, context) for rule in rules_list]))

In [ ]:
rules = [when_digit_rule]
rule_based_classification(df_ar_mini[0, "translation"], df_ar_mini[0, "context"], rules)

In [ ]:
trans_cont_tuples = [(trans, cont) for trans, cont in zip(df_ar_mini["translation"], df_ar_mini["context"])]

In [ ]:
answerable_preds = [rule_based_classification(t, c, rules) for t, c in trans_cont_tuples]
answerable_preds

In [ ]:
gt_answerables = df_ar_mini["answerable"].to_list()
score = sum([pred == gt for pred, gt in zip(answerable_preds, gt_answerables)]) / len(gt_answerables)
score